## 빈도분석

In [4]:
import pandas as pd

In [15]:
file = "./data/un-general-debates/un-general-debates-blueprint.csv"
df = pd.read_csv(file)
df.sample(2)

,session,year,country,country_name,speaker,position,text
4677,56,2001,CUB,Cuba,Felipe Pérez Roque,Minister for Foreign Affairs,"﻿Before delivering my statement, I would like ..."
3462,49,1994,HRV,Croatia,Mr. Franjo Tudjman,President,It is my\nparticular privilege and pleasure to...
